In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '663ff3650c35de4553dbd25b8eb2de0d'

In [57]:
movie_year = 2020

In [58]:
save_cleaned_data_path = 'D:/github-repos/web-crawler/data/02_intermediate/{}movies.csv'.format(movie_year)

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie()

In [7]:
def load_data():
    '''This function loads the wiki data from directory data/01_raw'''
    global data
    path = 'D:/github-repos/web-crawler/data/01_raw/wiki_data.csv'
    data = pd.read_csv(path)

In [8]:
load_data()

In [9]:
data.columns

Index(['Opening', 'Opening.1', 'Title', 'Production company', 'Cast and crew',
       '.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.',
       'Ref.'],
      dtype='object')

In [10]:
def extracting_features():
    '''This function extracts the features from our scraped dataframe'''
    global data 
    data = data[['Title', 'Cast and crew']]

In [11]:
extracting_features()

In [12]:
data.columns

Index(['Title', 'Cast and crew'], dtype='object')

In [15]:
def get_genre(x):
    '''This function introduces the lofic to get genres for each movie title by making an API call to the TMDB
    :param: title column from the scrapped wiki data'''
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [16]:
def make_genres_column():
    '''This function creates a new column called genres in our dataset with the genres for each movie'''
    data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

In [17]:
make_genres_column()

In [19]:
data['genres']

0             Horror Mystery Thriller
1       Action Horror Science Fiction
2                              Comedy
3                               Drama
4                Crime Thriller Drama
                    ...              
270      Action Fantasy Family Comedy
271    Drama Western Adventure Action
272                             Drama
273              Thriller Crime Drama
274                             Drama
Name: genres, Length: 275, dtype: object

In [18]:
def get_director(x):
    '''This function introduces the logic to filter directors from the scrapped wiki data
    :param: cast and crew column to filter directors from'''
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [20]:
def make_directors_column():
    '''This function gets directors from the cast and crew column of the scrapped data'''
    data['director_name'] = data['Cast and crew'].map(lambda x: get_director(str(x)))

In [21]:
make_directors_column()

In [22]:
data['director_name']

0         Nicolas Pesce
1        William Eubank
2         Miguel Arteta
3             Jon Avnet
4        Anthony Jerjen
             ...       
270    Robert Rodriguez
271     Paul Greengrass
272         Regina King
273     Emerald Fennell
274    Kornél Mundruczó
Name: director_name, Length: 275, dtype: object

In [23]:
def get_actor1(x):
    '''This function gets the principal actor in the movie
    :param: cast and crew column from data'''
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [24]:
def make_actor1_column():
    '''This function makes a new column with names of pricipal actors'''
    data['actor_1_name'] = data['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [26]:
make_actor1_column()

In [27]:
data['actor_1_name']

0         Andrea Riseborough
1            Kristen Stewart
2            Tiffany Haddish
3               Richard Gere
4              Josh Hartnett
               ...          
270    Priyanka Chopra Jonas
271                Tom Hanks
272        Kingsley Ben-Adir
273           Carey Mulligan
274            Vanessa Kirby
Name: actor_1_name, Length: 275, dtype: object

In [28]:
def get_actor2(x):
    '''This function gets the supporting actor in the movie
    :param: cast and crew column from data'''
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [29]:
def make_actor2_column():
    '''This function makes a new column with names of supporting actors'''
    data['actor_2_name'] = data['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [30]:
make_actor2_column()

In [31]:
data['actor_2_name']

0          Demián Bichir
1         Vincent Cassel
2             Rose Byrne
3         Peter Dinklage
4      Margarita Levieva
             ...        
270         Pedro Pascal
271        Helena Zengel
272            Eli Goree
273           Bo Burnham
274         Shia LaBeouf
Name: actor_2_name, Length: 275, dtype: object

In [32]:
def get_actor3(x):
    '''This function makes a new column with names of supporting actors'''
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [33]:
def make_actor3_column():
    '''This function makes a new column with names of supporting actors'''
    data['actor_3_name'] = data['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [34]:
make_actor3_column()

In [35]:
data['actor_3_name']

0             John Cho
1      Jessica Henwick
2          Salma Hayek
3       Walton Goggins
4       Chandler Riggs
            ...       
270      YaYa Gosselin
271                NaN
272        Aldis Hodge
273        Alison Brie
274       Molly Parker
Name: actor_3_name, Length: 275, dtype: object

In [36]:
data.columns

Index(['Title', 'Cast and crew', 'genres', 'director_name', 'actor_1_name',
       'actor_2_name', 'actor_3_name'],
      dtype='object')

In [39]:
def renaming_title_column():
    '''This function renames the title column to movie_title'''
    global data
    data = data.rename(columns={'Title':'movie_title'})

In [40]:
renaming_title_column()

In [41]:
data.columns

Index(['movie_title', 'Cast and crew', 'genres', 'director_name',
       'actor_1_name', 'actor_2_name', 'actor_3_name'],
      dtype='object')

In [42]:
def extract_features():
    '''This function extracts features to be used later in this project'''
    global cleaned_data
    cleaned_data = data.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [44]:
extract_features()

In [45]:
cleaned_data.columns

Index(['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name',
       'genres', 'movie_title'],
      dtype='object')

In [46]:
def combining_features():
    '''This function combines all the features into one column'''
    cleaned_data['comb'] = cleaned_data['actor_1_name'] + ' ' + cleaned_data['actor_2_name'] + ' '+ cleaned_data['actor_3_name'] + ' '+ cleaned_data['director_name'] +' ' + cleaned_data['genres']

In [47]:
combining_features()

In [48]:
cleaned_data.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      5
actor_3_name     28
genres            3
movie_title       0
comb             30
dtype: int64

In [51]:
def removing_nulls():
    global cleaned_data
    '''This function removes missing values from the dataset'''
    cleaned_data = cleaned_data.dropna(how='any')

In [52]:
removing_nulls()

In [53]:
cleaned_data.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [54]:
def make_movie_title_lowercase():
    '''This function makes all movie titles in the movie column lowercase'''
    cleaned_data['movie_title'] = cleaned_data['movie_title'].str.lower()

In [55]:
make_movie_title_lowercase()

In [56]:
cleaned_data['movie_title']

0                 the grudge
1                 underwater
2                like a boss
3              three christs
4          inherit the viper
               ...          
269                     soul
270         we can be heroes
272    one night in miami...
273    promising young woman
274        pieces of a woman
Name: movie_title, Length: 245, dtype: object

In [59]:
def save_the_processed_data():
    '''This function saves the process data to data/02_intermediate directory'''
    cleaned_data.to_csv(save_cleaned_data_path, index=False)

In [60]:
save_the_processed_data()